# Importing libraries

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import plotly.express as px
import os
import itertools
from modules import topology as tpl
from modules import postprocess as post
import pyvista as pv
import plotly
from IPython.display import display, HTML
import csv
import json
import pickle
import k3d
pio.renderers.default = "vscode"
template = "seaborn"

# Postprocessing tools
## 1D graphs
### TPB denity

In [ ]:
id = [82,83]

# Import libraries
fig = go.Figure()
template = 'seaborn'
# dx= [25e-9, 2e-9]
dx = [50e-9, 25e-9]

for i,id in enumerate(id):
    id = str(id).zfill(3)
    TPB_dict = np.load(f'Binary files/image analysis/scalars_{id}.npz')

    TPB_dist_x = TPB_dict['TPB_dist_x']
    x = TPB_dist_x.shape[0]

    fig.add_trace(
        go.Scatter(
            x=np.arange(x)*dx[i]*1e6, 
            y=TPB_dist_x/dx[i]*1e-12, 
            mode='markers', 
            name=f'ID: {id}',
            ))

    fig.update_layout(
        xaxis_title="Distance from anode/electrolyte interface [μm]",
        yaxis_title="TPB density in perpenducular plane [µm/µm<sup>3</sup>]",
        template=template
        )

# make it square
fig.update_layout(
    width=700,
    height=500,
    # set the xlim
    # xaxis=dict(
    #     range=[0, 50],
    #     ),
    autosize=False,
    hovermode=False,
    )
fig.show()
# pio.write_image(fig, f'Binary files/image analysis/TPB_dist.svg')


### Volume fraction

In [ ]:
ids = [82,83]
dx= [50e-9, 25e-9]

fig = go.Figure()
template = 'plotly_dark'

for id_idx, id in enumerate(ids):
    domain_file = np.load(f'Binary files/image analysis/domain_{str(id).zfill(3)}.npz')

    domain = domain_file['domain']
    vol_frac = [np.zeros(domain.shape[0]) for _ in range(3)]
    x = domain.shape[0]
    phases = ['Pores', 'Ni', 'YSZ']
    for p_idx, phase in enumerate([1,2,3]):
        for i in range(domain.shape[0]):
            vol_frac[p_idx][i] = np.sum(domain[i,...]==phase)/domain[i,...].size

        fig.add_trace(go.Scatter(
            x=np.arange(x)*dx[id_idx], 
            y=vol_frac[p_idx], 
            mode='lines', 
            name=f'ID: {id}, {phases[p_idx]}',
            ))
    

fig.update_layout(
    title="Variation of volume fraction along X direction",
    xaxis_title="X [m]",
    yaxis_title="Volume fraction [-]",
    template=template,
    width=1000,
    height=500,
    autosize=False,
    )
fig.show()
# save svg file
# pio.write_image(fig, f'Binary files/image analysis/vol_frac_{id}.svg')

### J-V curve

In [ ]:
# inputs
ids = [2]
# parameter = 'Ia_A'
entire = ['entire']*len(ids)
show_max_min = True
save_svg = False
compare_1D_model = False
col_pal = px.colors.qualitative.Bold

# allocate variables
n_ids = len(ids)
filename = [[None] * n_ids, [None] * n_ids]
filepath = [[None] * n_ids, [None] * n_ids]
df = [[None] * n_ids, [None] * n_ids]
max_x = [[None] * n_ids, [None] * n_ids]
fig = [[None] * n_ids, [None] * n_ids]
color = [None]*n_ids
col_pal_iterator = itertools.cycle(col_pal) 
cwd = os.getcwd()

# create the dataframes
for p_idx,parameter in enumerate(['Ia', 'Vio']):
    for id in range(n_ids):
        # name_str is id with three digits
        filename[p_idx][id] = parameter + '_' + entire[id] + '_' + str(ids[id]).zfill(3) + '.csv'
        
        filepath[p_idx][id] = os.path.join(
            cwd, 
            'Binary files', 
            '1D plots', 
            'csv', 
            filename[p_idx][id])
        
        df[p_idx][id] = pd.read_csv(filepath[p_idx][id])
        max_x[p_idx][id] = df[p_idx][id].iloc[-1,0] + df[p_idx][id].iloc[1,0] - df[p_idx][id].iloc[0,0]

if len(set(max_x[0])) > 1:
    for p_idx,parameter in enumerate(['Ia', 'Vio']):
        for id in range(n_ids): 
            df[p_idx][id].iloc[:,0] += max(max_x[0]) - max_x[0][id]
    
for id in range(n_ids):
    color[id] = next(col_pal_iterator)
    for p_idx,parameter in enumerate(['Ia', 'Vio']):
        x = df[p_idx][id].iloc[:,0]
        y = df[p_idx][id].iloc[:,1]
        y /= 1e4 if parameter == 'I' else 1          # convert to A/cm2

        fig[p_idx][id] = go.Scatter(
            x=x,
            y=y,
            name=filename[p_idx][id][:-4],
            mode='lines',
            line=dict(
                color=color[id], 
                width=2, 
                dash='solid' if parameter == 'Ia' else 'dash',
                ),
            yaxis='y1' if parameter == 'Ia' else 'y2',
            )

if show_max_min:
    for id in range(n_ids):
        x = df[1][id].iloc[:,0]
        y = df[1][id].iloc[:,2]
        fig[1].append(go.Scatter(
            x=x,
            y=y,
            line=dict(width=0),
            showlegend=False,
            ))
        
        color_RGB = px.colors.convert_colors_to_same_type(color[id], colortype='rgb')
        color_alpha = 'rgba(' + color_RGB[0][0][4:-1] + ', 0.2)'

        x = df[1][id].iloc[:,0]
        y = df[1][id].iloc[:,3]
        fig[1].append(go.Scatter(
            x=x,
            y=y,
            fillcolor = color_alpha,
            fill='tonexty',
            line=dict(width=0),
            showlegend=False))

# Add the traces to the figure
from modules.postprocess import create_fig_from_1D_model as cff1D
data = fig[0] + fig[1] + cff1D(next(col_pal_iterator),ids[0]) if compare_1D_model else fig[0] + fig[1]
fig = go.Figure(data)

# Set the x and y axes
fig.update_layout(
    xaxis_title = "Distance from anode side [µm]",
    yaxis_title = "Current density [A/cm<sup>2</sup>]",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'),
    template=template,
    width=600,
    height=500,
    legend=dict(x=0.65, y=0.99),
    font_family="Computer Modern",
    yaxis2=dict(
        title='Ion potential [V]',
        overlaying='y',
        side='right',
        ),
    xaxis=dict(
        range=[0, max(max_x[0])],
        ),
    )

fig.update_traces(
    hoverinfo = 'x+y',
)

# save svg file
if save_svg: fig.write_image('Vio_I.svg')

# Display the plot
fig.show()

### J-V curve (validation)

In [ ]:
V = np.array([      # Overpotential [V],
    0,
    -0.014929577,
    -0.022676056,
    -0.030140845,
    -0.034929577,
    -0.03943662,
    -0.046056338,
])

I_exp = np.array([      # Current density [A/cm2]
    0,
    0.0580,
    0.1099,
    0.1549,
    0.2027,
    0.2469,
    0.3096,
])

I_model_fitted = np.array([      # Current density [A/cm2]
    0,
    0.093336,
    0.14331,
    0.18722,
    0.21872,
    0.24400,
    0.28846,
])

I_model_upper = np.array([      # Current density [A/cm2]
    0,
    0.11395,
    0.17498,
    0.22861,
    0.26708,
    0.29797,
    0.35228,
])

I_model_lower = np.array([      # Current density [A/cm2]
    0,
    0.066271,
    0.10175,
    0.13291,
    0.15525,
    0.17319,
    0.20473,
])

V_model_3D = np.array([      # Overpotential [V]
    0,
    -0.0015327695560253714,
    -0.005084566596194506,
    -0.010000000000000004,
    -0.020052854122621564,
    -0.029883720930232567,
    -0.03996828752642706,
    -0.049767441860465125,
])

I_model_3D = np.array([      # Current density [A/cm2]
    0,
    0.0063178677196446265,
    0.02369200394866733,
    0.049555774925962484,
    0.10720631786771967,
    0.1699901283316881,
    0.24244817374136216,
    0.32082922013820325,
])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=I_model_upper, 
    y=V, 
    mode='lines', 
    name='Model Upper Bound',
    line=dict(
        color='rgba(127,60,141,1)',
        width=1,
        dash='dash',
    ),
    showlegend=False,
    ))

fig.add_trace(go.Scatter(
    x=I_model_fitted, 
    y=V, 
    mode='lines', 
    name='1D homogeneous model with <br> ± 50% change in calibration <br> parameter',
    fill='tonexty',
    fillcolor='rgba(127,60,141,0.1)',
    line=dict(
        color='rgba(127,60,141,1)',
        width=2,
    ),
    ))

fig.add_trace(go.Scatter(
    x=I_model_lower, 
    y=V, 
    mode='lines', 
    name='Model Lower Bound',
    fill = 'tonexty',
    fillcolor='rgba(127,60,141,0.1)',
    line=dict(
        # set color to 7f3c8dff
        color='rgba(127,60,141,1)',
        width=1,
        # dash line
        dash='dash',
    ),
    showlegend=False,
    ))

fig.add_trace(go.Scatter(
    x=I_model_3D, 
    y=V_model_3D, 
    mode='lines', 
    name='3D heterogeneous model',
    line=dict(
        # set color to 3969acff
        color='rgba(57,105,172,1)',
        width=2,
    ),
    ))

fig.add_trace(go.Scatter(
    x=I_exp, 
    y=V, 
    mode='markers', 
    name='Experimental Data',
    marker_symbol='x',
    marker=dict(
        size=10,
        # set color to 10a579ff
        color='rgba(16,165,121,1)',
    ),
    ))

fig.update_layout(
    # title="Overpotential vs. Current Density for different microstructures and different hydrogen partial pressures",
    xaxis_title="Current Density [A/cm<sup>2</sup>]",
    yaxis_title="Activation Overpotential [V]",
    # legend_title="title legend",
    template='seaborn',
    # legend overlap with the plot
    legend=dict(
        x=0.47,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='rgba(255, 255, 255, 0.5)',
        borderwidth=1,
    ),
    # width and height of the plot
    width=550,
    height=460,
    # set font
    font=dict(
        family="CMU serif",
        size=12,
        color="black"
    ),
)

# save pdf file
# pio.write_image(fig, "polarization.pdf")
# write html file
# fig.write_html("sdfs.html")
fig.show()

## 2D graphs
### Contours

In [2]:
# inputs
id = 2
entire = 'entire'
n_snapshots = 3
lattice_flag = False
N_tiles = 2
zmin = None
zmax = None
show_TPB = False
colormap = 'Viridis'


# file name
id = str(id).zfill(3)
# matrices = np.load(f'Binary files/arrays/matrices_{entire}_{id}.npz')
with open('test.pkl', 'rb') as f:
        matrices = pickle.load(f)
Vio = matrices['Vio']
TPB_dict = matrices['TPB_dict']
zmin = np.nanmin(Vio) if zmin is None else zmin
zmax = np.nanmax(Vio) if zmax is None else zmax

# adjust the number of snapshots
if type(n_snapshots) == str:
    if n_snapshots == 'all':
        n_snapshots = Vio.shape[1]
    else:
        raise Exception('n_snapshots should be an integer or "all"')
elif n_snapshots > Vio.shape[1]:
    n_snapshots = Vio.shape[1]


# adjust the shape of lattice
if lattice_flag: Vio = np.tile(Vio, [1,N_tiles,1])

# Create figure
fig = go.Figure()
# Add traces, one for each slider step [in the Z direction]
for i, step in enumerate(np.linspace(0, Vio.shape[2]-1, n_snapshots, dtype=int)):
    fig.add_trace(
        go.Heatmap(
            visible=False,
            z=np.rot90(Vio[:,:,step]),
            zmin=zmin,
            zmax=zmax,
            colorscale=colormap,
            ))
    # show TPB points as red dots
    if show_TPB:
        mask = np.rot90(TPB_dict['TPB_mask'][:,:,step]).astype(int)
        fig.add_trace(
            go.Scatter(
                x=np.where(mask==1)[1],
                y=np.where(mask==1)[0],
                mode='markers',
                marker=dict(
                    # rectangular shape marker
                    symbol='square',
                    color='red',
                    size=2,
                    ),
                visible=False,
                ))


# Make the first trace visible
fig.data[0].visible = True
if show_TPB: fig.data[1].visible = True

# Create and add slider
steps = []
if not show_TPB:
    for id in range(len(fig.data)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},
                {"title": "Slider switched to step: " + str(id)}],  # layout attribute
        )
        step["args"][0]["visible"][id] = True  # Toggle i'th trace to "visible"
        steps.append(step)
else:
    for id in range(0, len(fig.data)//2):
        step = dict(
            method="update",
            args=[{"visible": [False] * 2*len(fig.data)},
                {"title": "Slider switched to step: " + str(id)}],  # layout attribute
        )
        step["args"][0]["visible"][id*2] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][id*2+1] = True
        steps.append(step)

sliders = [dict(
    active=0,
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    template=template,
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1
    )

# save svg file
pio.write_image(fig, 'contour.svg')

fig.show()

### Roughness figures

In [ ]:
domain = tpl.create_microstructure_plurigaussian(
    voxels=[2000,2000],
    vol_frac=[0.4,0.3,0.3],
    d_ave=250,
    dx=2,
    seed=[1,65],
)

domain_rough = tpl.add_roughness_all_phases(
    domain,
    d_rough=20,
    iteration=3
)

domain_down = tpl.downscale_domain(
    domain,
    scale=16
)

post.plot_domain(
    [domain_down,domain_rough], 
    renderer='browser',
    template='seaborn',
    link_views=False,
)

## 3D graphs
### Volumetric data

In [ ]:
ids = [1,2]
entire = [True] * len(ids)
lattice_geometry = [False] * len(ids)
N_tiles = 10

rhoH2_flag   = False
Vel_flag     = False
Vio_flag     = True
rhoO2_flag   = False
I_flag       = False
eta_act_flag = False
eta_con_flag = False
domain_flag  = False

# gradients_flag = ['Z']
gradients_flag = False
isosurface_flag = False
TPB_flag     = True


# empty lists
mats, thds, log_scale, titles, TPB_mats, entire_mats = [], [], [], [], [], []
flag_vars = [rhoH2_flag, Vel_flag, Vio_flag, rhoO2_flag, I_flag, eta_act_flag, eta_con_flag, domain_flag]
vars =['rhoH2', 'Vel', 'Vio', 'rhoO2', 'Ia', 'eta_act', 'eta_con']

for i, id in enumerate(ids):
    if not entire[i]:
        flag_vars.remove(rhoO2_flag)
        vars.remove('rhoO2')
        
    # add zeros to the name of the file if the id is less than 100
    id = str(id).zfill(3)
    # matrices = np.load(f'Binary files/arrays/matrices_{sim[i]}_{id}.npz',allow_pickle=True)
    sim_type = 'entire' if entire[i] else 'cathode'
    with open(f'Binary files/arrays/matrices_{sim_type}_{id}.pkl', 'rb') as f:
        matrices = pickle.load(f)

    # create tiled mesh in y and z direction
    tiles = [1,1,1]
    if lattice_geometry[i]:
        tiles = [1,N_tiles,N_tiles]

    phi = np.tile(matrices['phi'], tiles)
    mask = matrices['mask']

    if TPB_flag:
        from modules.topology import create_vertices_in_uniform_grid as cvug
        if entire[i]:
            TPB_dict_a = matrices['TPB_dict']['anode']
            TPB_dict_c = matrices['TPB_dict']['cathode']

            TPB_mesh_a = pv.PolyData(
                cvug(mask[0].shape), 
                lines=TPB_dict_a['lines'],
                )

            vertices_c = cvug(mask[3].shape)
            vertices_c[:,0] += mask[1].shape[0] - mask[3].shape[0]
            TPB_mesh_c = pv.PolyData(
                vertices_c, 
                lines=TPB_dict_c['lines'],
                )
            TPB_mesh = [TPB_mesh_a, TPB_mesh_c]

        else:
            TPB_dict = matrices['TPB_dict']
            TPB_mesh = pv.PolyData(
                tpl.create_vertices_in_uniform_grid(phi.shape), 
                lines=TPB_dict['lines'],
                )

    for j,var in enumerate(vars):
        if flag_vars[j]:
            mats.append(np.tile(matrices[var], tiles))
            thds.append([])
            if TPB_flag: TPB_mats.append(TPB_mesh)
            entire_mats.append(entire[i])
            # log_scale.append(True if var == 'I' else False)
            log_scale.append(False)
            titles.append(f'{var}_{id}')

if not isosurface_flag:
    # plt_voxels = k3d.volume(
    #     mats[0],
    #     bounds=[1, mats[0].shape[2], 1, mats[0].shape[1], 1, mats[0].shape[0]],
    #     alpha_coef=10000,
    # )
    # plot = k3d.plot()
    # plot += plt_voxels
    # plot.display()

    post.visualize_mesh(
        mat = mats,
        thd = thds,
        titles = titles,
        # clip_widget = True, 
        TPB_mesh = TPB_mats if TPB_flag else None,
        entire_domain = entire_mats,
        log_scale = log_scale,
        # elevation = 20,
        # azimuth = 60,
        link_views = True,
        # isometric=True,
        # save_graphics=False,
        show_axis = True,
        # link_colorbar=True,
        notebook = False,
        # save_html=True,
        # background_color='black',
        )
else:
    post.visualize_contour(Vio,n_levels=10)

### TPB

In [ ]:
ids = [999]
TPB_mask = [None]*len(ids)
for id_idx, id in enumerate(ids):
    domain = np.load(f'Binary files/image analysis/domain_{str(id).zfill(3)}.npz')
    TPB_mask[id_idx] = domain['TPB_mask']

post.visualize_mesh(
    [TPB_mask[0],TPB_mask[1]],      # ??? 
    link_views=False,
    )

### Volumteric data and TPB

In [ ]:
id = 50
id = str(id).zfill(3)
matrices = np.load(f'Binary files/arrays/matrices_{id}.npz')
tiles = [1,1,1]

Vio = np.tile(matrices['Vio'],tiles)
I = np.tile(matrices['I'],tiles)

N=Vio.shape

p = pv.Plotter(
    notebook=False, 
    off_screen=False
    )

mesh_Vio = pv.ImageData(dimensions=(N[0]+1,N[1]+1,N[2]+1))
mesh_Ia = pv.ImageData(dimensions=(N[0]+1,N[1]+1,N[2]+1))

mesh_Vio.cell_data['Vio'] = Vio.T.flatten()
mesh_Ia.cell_data['I'] = I.T.flatten()

mesh_Vio = mesh_Vio.threshold()
mesh_Ia = mesh_Ia.threshold()

sargs1 = dict(
    height=0.25, 
    vertical=True, 
    position_x=0.05, 
    position_y=0.05)

sargs2 = dict(
    height=0.25, 
    vertical=True, 
    position_x=0.2,  
    position_y=0.05)

# Ion potential
p.add_mesh(
    mesh_Vio, 
    cmap='viridis', 
    # show_scalar_bar=True, 
    scalar_bar_args=sargs1,
    lighting=True,
    log_scale=False)

# Current density
p.add_mesh(
    mesh_Ia,
    cmap='Reds',
    # show_scalar_bar=True,
    scalar_bar_args=sargs2,
    lighting=False,
    log_scale=True)

p.camera_position = 'xy'
p.camera.azimuth = 90
p.camera.roll = 45
p.camera.azimuth = 45
p.camera.clipping_range = (1e-2, 1e9)
p.add_bounding_box(line_width=1, color='black')
p.show()
# scale_factor = 2
# p.window_size = [scale_factor*p.window_size[0],scale_factor*p.window_size[1]]
# p.screenshot('test2.png')

### Gradients

In [ ]:
# Don't use this function to find the gradients. 
# Gradients should be calculated from the Jacobian matrix itself.

id = 3

from findiff import FinDiff
dx = 50e-9
d_dx = FinDiff(0, dx, acc=2)
T = 1073

id = str(id).zfill(3)
matrices = np.load(f'Binary files/arrays/matrices_entire_{id}.npz')

Vio = matrices['Vio']
dVio_dx = d_dx(Vio)
# dVio_dx = np.gradient(Vio, dx, axis=0)
# dVio_dx = Vio[:-1,...] - Vio[1:,...]

fluxion_matrix = np.zeros_like(dVio_dx)*np.nan

cond_ion_a = 3.34e4 * np.exp(-10350/T)    # [S/m]
cond_ion_electrolyte = 4e-1  # ??? [S/m] @ T=1173.15 https://doi.org/10.1016/j.ssi.2004.11.019
cond_ion_c = 4e-1 # [S/m] @ T=1173.15 https://doi.org/10.1016/j.ssi.2004.11.019

K_mat = np.zeros_like(Vio)*np.nan
K_mat[:200,...] = cond_ion_a
K_mat[200:220,...] = cond_ion_electrolyte
K_mat[220:,...] = cond_ion_c
K_mat[np.isnan(Vio)] = np.nan

# fluxion_matrix = - K_mat * dVio_dx * Area [A]
fluxion_matrix[:200,...]    = - K_mat[:200,...]     * dVio_dx[:200,...]     * 0.3
fluxion_matrix[200:220,...] = - K_mat[200:220,...]  * dVio_dx[200:220,...]  * 1
fluxion_matrix[220:,...]    = - K_mat[220:,...]     * dVio_dx[220:,...]     * 0.3

fluxion_mean = np.nanmean(fluxion_matrix, axis=(1,2))
fluxion_max = np.nanmax(fluxion_matrix, axis=(1,2))
fluxion_min = np.nanmin(fluxion_matrix, axis=(1,2))

from modules.postprocess import plot_with_continuous_error as ploterr
ploterr(
    x=np.arange(fluxion_matrix.shape[0])*dx,
    y=fluxion_mean,
    y_min=fluxion_min,
    y_max=fluxion_max,
)

### TPB schematic visualization

In [ ]:
a = np.array([[1, 1], [1, 1]])
b = np.array([[1, 1], [2, 3]])
c = np.stack((a, b), axis=0)

mask, density, lines, dist = tpl.measure_TPB(c,1)
TPB_mesh = pv.PolyData(
    tpl.create_vertices_in_uniform_grid(c.shape), 
    lines=lines,
    )

post.visualize_mesh(
    [mask],
    [[2,3]],
    TPB_mesh=[TPB_mesh],
    entire_domain=[False],
    edge_width=5,
    isometric=True,
    # clip_widget=True,
    cmap='Accent',
    link_colorbar=True,
    # opacity=0.5,
    # save_graphics=True,
    link_views=True,
    notebook=True,
    )

## Histograms
### Histogram of pore/particle size

In [ ]:
ids = [201,202,203,204]
Nhist = 200
template = 'plotly_dark'
all_phases = False
Violin = True

maxt = np.zeros(len(ids))
radius = [None] * len(ids)
for i,id in enumerate(ids):
    id = str(id).zfill(3)
    domain = np.load(f'Binary files/image analysis/scalars_{id}.npz')
    volumes = [None]*3
    radius[i] = [None]*3
    volumes[0] = domain['volumes_0']
    volumes[1] = domain['volumes_1']
    volumes[2] = domain['volumes_2']
    
    radius[i][0] = (volumes[0] / np.pi * 3/4)**(1/3)    # [μm]  pores
    radius[i][1] = (volumes[1] / np.pi * 3/4)**(1/3)    # [μm]  Ni
    radius[i][2] = (volumes[2] / np.pi * 3/4)**(1/3)    # [μm]  YSZ

    # print the average diameter
    print(f'mean diameter of pores in ID {id}: {1000*2*np.median(radius[i][2][:,0]):.2f} nm')
    max0 = np.max(volumes[0])
    max1 = np.max(volumes[1])
    max2 = np.max(volumes[2])
    maxt[i] = np.max([max0,max1,max2])

maxt = np.max(maxt) / 1
fig = go.Figure()
for i,id in enumerate(ids):
    fig.add_trace(go.Violin(
        x=2*radius[i][2][:,0], 
        # xbins=dict( # bins used for histogram
        #     start=0,
        #     end=maxt,
        #     size=maxt/Nhist),
        name=f'YSZ {id}',
        # histnorm='probability',
            ))

    if all_phases:
        fig.add_trace(go.Violin(
            x=2*radius[i][0][:,0], 
            xbins=dict( # bins used for histogram
                start=0,
                end=maxt,
                size=maxt/Nhist),
            name=f'Pores {id}',
            histnorm='probability',
                ))

        fig.add_trace(go.Violin(
            x=2*radius[i][1][:,0], 
            xbins=dict( # bins used for histogram
                start=0,
                end=maxt,
                size=maxt/Nhist),
            name=f'Ni {id}',
            histnorm='probability',
            ))

# Overlay both histograms
fig.update_layout(
    # barmode='overlay', 
    template=template,
    )

fig.update_traces(
    opacity=0.5 if all_phases or len(ids)>1 else 1,
    orientation='h', 
    side='positive', 
    width=3,
    # hovertemplate=None,
    hoverinfo='skip',
    )
fig.show()

## Boxcharts
### Boxchart TPB (rough - smooth)

In [ ]:
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
 
# read the excel file
file = "C:\\Users\\x67637ha\\OneDrive - The University of Manchester\\SOFC\\Reports\\xlsx\\topology_investigation.xlsx"
df = pd.read_excel(file, sheet_name = 'new_dx_PG_only')
 

# make a box plot
template = 'seaborn'
# fig = go.Figure()
fig = px.box(
    df, 
    x='_dx', 
    y='_TPBd', 
    # color='Algorithm', 
    points='all', 
    log_x=True, 
    log_y=False,
    template=template)

# D1 = 1.55
# D2 = 1.61
# z = np.linspace(5,40,100)
# L1 = 150*z**(1-D1)
# L2 = 20*z**(1-D2)

# D1_0 = 1.27
# z_0 = np.linspace(1.5625,50,100)
# L1_0 = 13*z_0**(D1_0-1)

# D1_1 = 0.95
# z_1 = np.linspace(1.5625,50,100)
# L1_1 = 21.5*z_1**(D1_1-1)

# D_quad = 1.67
# z_quad = np.linspace(1.5625,50,100)
# L_quad = 3.9*z**(D_quad-1)

# D_Koch = 1.2619
# z_Koch = np.linspace(1.5625,50,100)
# L_Koch = 11.8*z_Koch**(D_Koch-1)


# fig.add_trace(go.Scatter(
#     x=z,   
#     y=L1,   
#     mode='lines', 
#     line = dict(color='black', width=1, dash='dash'), 
#     showlegend=True,
#     name='D = 1.55'))
# fig.add_trace(go.Scatter(x=z,   y=L2,   mode='lines', line = dict(color='black', width=1, dash='solid'), showlegend=False))
# fig.add_trace(go.Scatter(x=z_0, y=L1_0, mode='lines', line = dict(color='gray', width=1, dash='dot'), showlegend=False))
# fig.add_trace(go.Scatter(x=z_1, y=L1_1, mode='lines', line = dict(color='gray', width=1, dash='dot'), showlegend=False))
# fig.add_trace(go.Scatter(
#     x=z_quad, 
#     y=L_quad, 
#     mode='lines', 
#     line = dict(color='red', width=1, dash='dash'), 
#     showlegend=True,
#     name='Quadratic'))
# fig.add_trace(go.Scatter(
#     x=z_Koch, 
#     y=L_Koch, 
#     mode='lines', 
#     line = dict(color='blue', width=1, dash='dash'), 
#     showlegend=True,
#     name='Koch snowflake'))
fig.update_layout(
    # title="variation of TPB density as a function of voxel size",
    # xaxis_title="Average feature size / Measurement unit",
    xaxis_title='Voxel size [nm]',
    yaxis_title='TPB density [µm/µm<sup>3</sup>]',
    # yaxis=dict(
    #     visible=False,),
    # yaxis_title='Normalized length of topological feature',
    width=700,
    height=500,
    autosize=True,
    template=template,
    legend=dict(x=0.75,y=0.95)
    )

# set the font family to computer modern
fig.update_layout(
    font_family="Computer Modern",
    # font_size="black",
    # title_font_family="Computer Modern",
    # title_font_color="black",
    # legend_title_font_color="black"
    )
fig.show()
pio.write_image(fig, f'Binary files/image analysis/dx on TPBd.svg')

### Boxchart TPB-current density

In [ ]:
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
 
# current density data from simulations
fG = np.array([-0.6,-0.4,-0.2,0,0.2,0.4,0.6])
I = np.array([5.145356462,5.308537465,4.883389893,4.133952018,3.159094341,2.064987341,0.811201492])

# read the excel file
file = "C:\\Users\\x67637ha\\OneDrive - The University of Manchester\\SOFC\\Reports\\xlsx\\topology_investigation.xlsx"
df = pd.read_excel(file, sheet_name = 'gradients')

template = 'seaborn'

fig = go.Figure()

# box chart for TPB density
fig.add_trace(go.Box(
    x=df['fG'],
    y=df['TPBd'], 
    boxpoints='all',
    name='TPB density',
    # points='all',
    # template=template,
    ))

# scatter plot for current density
fig.add_trace(go.Scatter(
    x=fG, 
    y=I, 
    mode='lines+markers', 
    name='Current density', 
    yaxis='y2',
    ))

fig.update_layout(
    xaxis_title='Gradient factor, f<sub>G</sub> [-]',
    yaxis_title='TPB density [µm/µm<sup>3</sup>]',
    width=600,
    height=500,
    # autosize=True,
    xaxis=dict(
        tickmode='array',
        tickvals=np.linspace(-0.6,0.6,7),
    ),
    template=template,
    legend=dict(x=0.7,y=0.95),
    font_family="Computer Modern",
    hovermode=False,
    yaxis2=dict(
        title='Current density [A/cm<sup>2</sup>]',
        overlaying='y',
        side='right',
        range=[0,6],
        ),
    )
fig.show()
pio.write_image(fig, f'Binary files/image analysis/gradient_TPB_Ia.svg')

# File management
## Read CSV files

In [ ]:
data = []
for id in range(998,1000):
    filestr = str(id).zfill(3)
    # file_name = 'Binary files\\1D plots\\csv\\latest\\Ia_A_' + filestr + '.csv'
    file_name = 'Binary files\\1D plots\\csv\\Ia_A_' + filestr + '.csv'
    df = pd.read_csv(file_name)
    property = 'Ia_A_' + filestr + '_avg'
    data.append(df[property].iloc[-2])

# write the list to a file
with open('outputs\output.txt', 'w') as f:
    for line in data:
        f.write(str(line) + '\n')

## Read CSF3 files

In [ ]:
# read files named job.txt.o4465196.1 to job.txt.o4465196.80 from the folder /outputs

data = []
for id in range(501,521):
    file_name = f'outputs\job.txt.o4529240.{id}'
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    # read the third line of the file
    line = lines[3]
    # append the line to the list
    data.append(line)

# write the list to a file
with open('outputs\output.txt', 'w') as f:
    for line in data:
        f.write(line)


## Write input files

In [ ]:
id_start = 7
n_cases = 1

# create a set of random integers between 1 and 100
# import random
# random_ints = [None] * 4
# for i in range(4):
#     random.seed(i)
#     random_ints[i] = random.sample(range(1, 100), n_cases)

params = {
    # "pH2_inlet": [0.95] * n_cases,
    # "average_diameter": [250e-9, 500e-9, 750e-9, 1000e-9],
    # "Vel_b": [0.01, 0.1, 0.2, 0.5],
    # "length_X": [10e-6]*5 + [20e-6]*5 + [40e-6]*5 + [50e-6]*5 + [100e-6]*5 + [10e-6]*5 + [20e-6]*5 + [30e-6]*5 + [40e-6]*5 + [100e-6]*5,
    # "scale_factor": [1,2,4,8,16],
    # "length_YZ": [10e-6] * n_cases,
    # "lattice_geometry": [False] * n_cases,
    # "vf_pores": [0.9,0.9,0.9,0.9,0.9,0.8,0.8,0.8,0.8,0.8,0.7,0.7,0.7,0.7,0.7,0.6,0.6,0.6,0.6,0.6,0.5,0.5,0.5,0.5,0.5,0.4,0.4,0.4,0.4,0.4],
    # "vf_Ni": [0.05,0.05,0.05,0.05,0.05,0.1,0.1,0.1,0.1,0.1,0.15,0.15,0.15,0.15,0.15,0.2,0.2,0.2,0.2,0.2,0.25,0.25,0.25,0.25,0.25,0.3,0.3,0.3,0.3,0.3],
    # "vf_YSZ": [0.05,0.05,0.05,0.05,0.05,0.1,0.1,0.1,0.1,0.1,0.15,0.15,0.15,0.15,0.15,0.2,0.2,0.2,0.2,0.2,0.25,0.25,0.25,0.25,0.25,0.3,0.3,0.3,0.3,0.3],
    # "gradient_factor": [0.0] * n_cases,
    # "sigma_segmentation": [2,3,4,5],
    # "upscale_factor": [1,2,4],
    # "dx": [12.5, 25, 50, 100],
    # "flux_Z_constant": [0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6]
}

f = open('input files/inputs_entire_002.json')
inputs = json.load(f)

for i in range(n_cases):
    id_num = id_start + i
    id_num = str(id_num).zfill(3)
    id_str = "input files/inputs_entire_" + id_num + ".json"
    inputs['file_options']['id'] = id_num

    # inputs['boundary_conditions']['pH2_inlet'] = params['pH2_inlet'][i]
    # inputs['boundary_conditions']['pH2_b'] = params['pH2_inlet'][i]
    # inputs['boundary_conditions']['Vel_b'] = params['Vel_b'][i]
    # inputs['microstructure']['scale_factor'] = params['scale_factor'][i]
    # inputs['microstructure']['lattice_geometry']['flag'] = params['lattice_geometry'][i]
    # inputs['microstructure']['length']['X'] = params['length_X'][i]
    # inputs['microstructure']['length']['Y'] = params['length_YZ'][i]
    # inputs['microstructure']['length']['Z'] = params['length_YZ'][i]
    # inputs['microstructure']['average_diameter'] = params['average_diameter'][i]
    # inputs['microstructure']['sigma_segmentation'] = params['sigma_segmentation'][i]
    # inputs['microstructure']['volume_fractions']['pores'] = params['vf_pores'][i]
    # inputs['microstructure']['volume_fractions']['Ni'] = params['vf_Ni'][i]
    # inputs['microstructure']['volume_fractions']['YSZ'] = params['vf_YSZ'][i]
    # inputs['microstructure']['plurigaussian']['gradient_factor'] = params['gradient_factor'][i]
    # inputs['microstructure']['anode']['plurigaussian']['seed'] = [random_ints[0][i], random_ints[1][i]]
    # inputs['microstructure']['cathode']['plurigaussian']['seed'] = [random_ints[2][i], random_ints[3][i]]
    # inputs['microstructure']['upscale_factor'] = params['upscale_factor'][i]
    # inputs['boundary_conditions']['flux_Z_constant'] = params['flux_Z_constant'][i]

    json_object = json.dumps(inputs, indent = 4)
    with open(id_str, "w") as outfile:
        outfile.write(json_object)

In [91]:
import plotly.graph_objects as go
import numpy as np

r1 = 1
r2 = 1
r3 = 1.25

x1 = np.linspace(0.28, 0.31, 50)
x2 = np.linspace(0.28, 0.31, 50)

xx1, xx2 = np.meshgrid(x1,x2)

TPB_inv = np.log((r1**3/xx1 + r2**3/xx2 + r3**3/(1-xx1-xx2)))
# TPB_inv = (r1/xx1 + r2/xx2 + r3/(1-xx1-xx2))

fig = go.Figure()
fig.add_trace(go.Contour(
    z=TPB_inv,
    x=x1,
    y=x2,
    showscale=False  # This line removes the colorbar
    ))

zmin = np.nanmin(TPB_inv)
loc = np.argmin(TPB_inv)
x_min = x1[loc%50]
y_min = x2[loc//50]

# add the minimum point
fig.add_trace(go.Scatter(
    x=[x_min], 
    y=[y_min], 
    mode='markers',
    marker=dict(
        size=5,
        color='red',
        ),
    ))

# add annotation for the minimum point
fig.add_annotation(
    x=x_min,
    y=y_min,
    text=rf'$\varepsilon_1={x_min:.2f}, \varepsilon_2={y_min:.2f}$',
    font=dict(color='white'),
    )

fig.update_layout(
    width= 400,
    height=400,
    xaxis_title=r'$\varepsilon_1$',
    yaxis_title=r'$\varepsilon_2$',
    template='simple_white',
    )

# save svg file
pio.write_image(fig, 'TPB_inv_2.pdf')

fig.show()


In [ ]:
from modules.topology import create_microstructure_plurigaussian as cmp
from modules.topology import create_vertices_in_uniform_grid as cvug
from modules.topology import measure_TPB as mtpb
import pyvista as pv
from modules.postprocess import visualize_mesh as vm

# create a microstructure
a = cmp([150,150,150], [0.3,0.3,0.4], 500e-9, 50e-9)
TPB_mask, TPB_density, lines, _ = mtpb(a,50e-9)
TPB_mesh_a = pv.PolyData(
    cvug(a.shape), 
    lines=lines,
    )
vm([a],[[3,3]],TPB_mesh=[TPB_mesh_a])

In [27]:
# import mat file into numpy array
import scipy.io
import numpy as np
vm = post.visualize_mesh
from modules.topology import create_vertices_in_uniform_grid as cvug
from modules.topology import measure_TPB as mtpb


mat = scipy.io.loadmat('hadi microstructures\\38.mat')

values = mat.values()
type(values)

# get the values from dict_values
values = list(values)
mat = values[-1]

# measure TPB
TPB_mask, TPB_density, lines, _ = mtpb(mat,50e-9)
TPB_mesh_a = pv.PolyData(
    cvug(mat.shape), 
    lines=lines,
    )

vm(
    [mat],
    [[4,4]],
    TPB_mesh=[TPB_mesh_a],
    cmap = 'plasma',
    )



KeyboardInterrupt: 